In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").appName('test').getOrCreate()

22/07/23 17:42:08 WARN Utils: Your hostname, kuchoco-GP62-6QE resolves to a loopback address: 127.0.1.1; using 192.168.219.113 instead (on interface wlp2s0)
22/07/23 17:42:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/07/23 17:42:09 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/23 17:42:10 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [5]:
df_green = spark.read.parquet('data/report/green/*.parquet')
df_green.registerTempTable('green')

df_yellow = spark.read.parquet('data/report/yellow/*.parquet')
df_yellow.registerTempTable('yellow')

In [13]:
df_green_tmp = df_green\
    .withColumnRenamed('amount', 'green_amount')\
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_tmp = df_yellow\
    .withColumnRenamed('amount', 'yellow_amount')\
    .withColumnRenamed('number_records', 'yellow_number_records')

## Join Two large table
- green and yellow taxi-trip data

In [15]:
df_join = df_green_tmp.join(df_yellow_tmp, on=['hour', 'zone'], how='outer')

In [16]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00| 141|              null|                null|             15.96|                    1|
|2020-01-01 10:00:00|  24|126.41999999999999|                   6| 783.8999999999999|                   44|
|2020-01-01 10:00:00| 242|             30.94|                   2|              null|                 null|
|2020-01-01 13:00:00| 202|              null|                null|              42.1|                    2|
|2020-01-01 14:00:00|  71|              null|                null|             49.71|                    1|
|2020-01-01 14:00:00| 244|             25.26|                   2|185.17000000000004|                   17|
|2020-01-01 16:00:00|  48|  

In [17]:
df_join.write.parquet('data/report/total', mode='overwrite')

## Join one large table and one small table
- taxi-trips and zones

no shuffling because zone data distributed to executors

In [18]:
df_zones = spark.read.parquet('zones/')

In [20]:
df_zones.show()

+----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly

In [23]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.locationid)

In [39]:
df_result.show(10)

+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|locationid|  borough|                zone|service_zone|
+-------------------+----+------------------+--------------------+------------------+---------------------+----------+---------+--------------------+------------+
|2020-01-01 00:00:00| 141|              null|                null|             15.96|                    1|       141|Manhattan|     Lenox Hill West| Yellow Zone|
|2020-01-01 10:00:00|  24|126.41999999999999|                   6| 783.8999999999999|                   44|        24|Manhattan|        Bloomingdale| Yellow Zone|
|2020-01-01 10:00:00| 242|             30.94|                   2|              null|                 null|       242|    Bronx|Van Nest/Morris Park|   Boro Zone|
|2020-01-01 13:00:00| 

In [35]:
# drop zone column(int) and save
df_result.drop(df_join.zone).repartition(10).write.parquet('tmp/revenue-zones/', mode ='overwrite')

22/07/23 19:05:40 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
